In [35]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall bitsandbytes
!pip -qqq install bitsandbytes

In [3]:
!pip install cuda-python
!pip install -qqq torch
# !pip install -qqq bitsandbytes
!pip install -qqq -U git+https://github.com/huggingface/transformers.git
!pip install -qqq -U git+https://github.com/huggingface/peft.git
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git
!pip install -qqq datasets
!pip install -qqq loralib
!pip install -qqq einops


# !git clone https://github.com/timdettmers/bitsandbytes.git
# !CUDA_VERSION=122 make cuda12x
# !python bitsandbytes/setup.py install

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 41.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [1]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
notebook_login()

# LOAD FALCON MODEL & TOKENIZER

In [6]:
models_to_try = ["mistralai/Mistral-7B-Instruct-v0.2",
                 "meta-llama/Llama-3.1-8B-Instruct"]

MODEL_NAME = models_to_try[0]

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [8]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [9]:
print(model)


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNo

In [10]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"],  # Updated target modules , query , key , value
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 9437184 || all params: 3761508352 || trainables%: 0.250888290464176


# Test original model

In [11]:
# prompt = """
# <human>: midjourney prompt for a girl sit on the mountain
# <assistant>:
# """.strip()

prompt = """who is the goat of soccer ?""".strip()

In [12]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.1
generation_config.top_p = 0.5
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

<h4> Testing the model before strating the finetuning

In [13]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


who is the goat of soccer ?

The title of the "Greatest of All Time" (GOAT) in soccer is a subject of much debate among fans and experts. Some popular contenders for the title include Lionel Messi, Cristiano Ronaldo, and Pelé.

Lionel Messi is widely regarded as one of the greatest soccer players of all time. He has won numerous awards, including six Ballon d'Or titles, and has led FC Barcelona to numerous domestic and international titles.

Cristiano Ronaldo is also considered one of the greatest soccer players of all time. He has won numerous awards, including five Ballon d'Or titles, and has led Real Madrid and Manchester United to numerous domestic and international titles.

Pelé is often considered the first GOAT of soccer. He won three World Cup titles with Brazil and is the all-time leading goalscorer in the history of the Brazilian national team.

Ultimately
CPU times: user 25.4 s, sys: 628 ms, total: 26.1 s
Wall time: 31.8 s


# Prep dataset

In [25]:
from datasets import DatasetDict
data = load_dataset("csv", data_files="data.csv")

In [26]:
data

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 26931
    })
})

## We only take 2000 traning samples to just to test the traning process, since the full dataset will take ~10 hours of traning

In [27]:
from datasets import DatasetDict

data = DatasetDict({
    "train": data["train"].select(range(2000))  # Selects the first 2000 rows
})

In [28]:
data

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 2000
    })
})

In [32]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["question"]}
<assistant>: {data_point["answer"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [33]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [34]:
data

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 2000
})

# Finetune the model

In [36]:
training_args = transformers.TrainingArguments(
      save_steps = 100,
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=1,
      logging_steps=1,
      output_dir="/content/drive/MyDrive/medLLM",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:500: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_aut

Step,Training Loss
1,3.281600
2,3.062500
3,3.318100
4,3.163700
5,2.502700
6,3.021800
7,2.981300
8,2.642500
9,2.595000
10,2.254500


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

TrainOutput(global_step=500, training_loss=1.4163579252958298, metrics={'train_runtime': 3070.289, 'train_samples_per_second': 0.651, 'train_steps_per_second': 0.163, 'total_flos': 6349560352358400.0, 'train_loss': 1.4163579252958298, 'epoch': 1.0})

# Save trained model

In [37]:
model.save_pretrained("trained-model-2000-samples")

In [40]:
trainer.push_to_hub(model_name="Mistral_7B_0.2_dermatology_2000_samples")

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1727640643.9dda75819818.307.0:   0%|          | 0.00/111k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/37.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AbdelMounaim/medLLM/commit/7aa403a04330924ad70d3e01a5f0078f8d128201', commit_message='End of training', commit_description='', oid='7aa403a04330924ad70d3e01a5f0078f8d128201', pr_url=None, pr_revision=None, pr_num=None)

In [41]:
PEFT_MODEL = "AbdelMounaim/Mistral_7B_0.2_dermatology_2000_samples"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)
tokenizer.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/37.8M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AbdelMounaim/Mistral_7B_0.2_dermatology_2000_samples/commit/8f19f3a49ae72cac1f3765e7f0df3b18b8d4737f', commit_message='Upload tokenizer', commit_description='', oid='8f19f3a49ae72cac1f3765e7f0df3b18b8d4737f', pr_url=None, pr_revision=None, pr_num=None)

In [42]:
from transformers.hf_api import HfApi

HfApi().delete_repo(PEFT_MODEL)

ModuleNotFoundError: No module named 'transformers.hf_api'

In [ ]:
PEFT_MODEL = "AbdelMounaim/Mistral_7B_0.2_dermatology_4800_samples"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/37.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AbdelMounaim/Mistral_7B_0.2_dermatology_4800_samples/commit/16f76ea31d137a20b40ea814eca3bf70384a65a4', commit_message='Upload model', commit_description='', oid='16f76ea31d137a20b40ea814eca3bf70384a65a4', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

# Run the finetuned model

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.1
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt = """
<human>: what is the strcuture of the skin ?
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve 

<human>: what is the strcuture of the skin ?
<assistant>: The skin is a complex organ that consists of three main layers: the epidermis, the dermis, and the subcutaneous tissue. The epidermis is the outermost layer of the skin, while the dermis is the middle layer. The subcutaneous tissue is the deepest layer. The skin also contains various glands, such as sweat glands and sebaceous glands, as well as hair follicles and sweat ducts. The skin is also rich in blood vessels and lymphatic vessels, which help to regulate body temperature and provide nutrients to the skin. The skin is also rich in nerve endings, which help to transmit sensations to the brain. The skin is also rich in immune cells, which help to protect the body from infection and disease. The skin is also rich in melanocytes, which produce melanin, a pigment that helps to protect the skin from the sun. The
CPU times: user 2min 22s, sys: 1min, total: 3min 23s
Wall time: 3min 29s


# Loading the saved model from huggingfece

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel



bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token = tokenizer.eos_token



# Load the fine-tuned LoRA adapter from your Hugging Face repo
adapter_model_name = "AbdelMounaim/Mistral_7B_0.2_dermatology_2000_samples"
model = PeftModel.from_pretrained(base_model, adapter_model_name)

# The model and tokenizer are now ready for inference!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/37.8M [00:00<?, ?B/s]

In [4]:
# Define a test prompt
prompt = "What are the common symptoms of skin cancer?"

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Ensure that it runs on GPU

# Generate output
outputs = model.generate(inputs.input_ids, max_new_tokens=100)

# Decode and print the generated text
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


What are the common symptoms of skin cancer?

What are the common symptoms of skin cancer?

Common symptoms of skin cancer include:

* A new mole or a change in an existing mole
* A sore that doesn't heal
* A lump or a thickening of the skin
* A change in the color, size, or shape of a birthmark
* A patch of skin that is rough, scaly, or itchy
* A sore that bleeds easily or doesn't heal
